# Amazon Bedrock Claude usecases for Indian Languages

*This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

Contents
- Claude Model Selection    
- Hindi Examples
    - Question Answering
    - Summarization
    - Translate
    - Transliterate
- Telugu Examples
    - Question Answering
    - Summarization
    - Translate
    - Transliterate

---
## Claude Model Selection

We have following Claude 3 models available as of today on Amazon Bedrock

| Model Name | Model ID |
|--|--|
| Claude 3 Haiku | anthropic.claude-3-haiku-20240307-v1:0 |
| Claude 3 Sonnet | anthropic.claude-3-sonnet-20240229-v1:0 |

- [Anthropic Claude Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html#model-parameters-anthropic-claude-messages-overview)
- [Create a Message](https://docs.anthropic.com/claude/reference/messages_post)
- [Amazon Bedrock API for Claude](https://docs.anthropic.com/claude/reference/claude-on-amazon-bedrock)
- [Amazon Bedrock Pricing Page](https://aws.amazon.com/bedrock/pricing/)
---

In [2]:
claude3_haiku_id = "anthropic.claude-3-haiku-20240307-v1:0"
claude3_sonnet_id = "anthropic.claude-3-sonnet-20240229-v1:0"

DEFAULT_MODEL = claude3_sonnet_id

In [3]:
import boto3
import json

class LLM:
    def __init__(self, model_id):
        self.model_id = model_id
        self.bedrock = boto3.client(service_name="bedrock-runtime")
        
    def invoke(self, system, messages, max_tokens=128):
        body = json.dumps({
                  "max_tokens": max_tokens,
                  "system": system,
                  "messages": messages,
                  "anthropic_version": "bedrock-2023-05-31"
                })
        response = self.bedrock.invoke_model(
                                            body=body, 
                                            modelId=self.model_id)

        response_body = json.loads(response.get("body").read())
        return response_body['content']
    
llm = LLM(DEFAULT_MODEL)

In [4]:
def system_prompt(language, task= 'qna'):
    sys_prompt = f"Hello, You are an expert in {language}, an Indian language."
    if task == 'qna':
        sys_prompt = sys_prompt + f"""\nGiven the content in {language} wrapped between <{language}> and </{language}> tags, there will be follow up questions asked to you in English between <question> and </question> tags. You respond in {language} language. You respond to the point and not verbose. Your response will be between the tags <response> and </response>. For responses that requires to identify entities, you just include those entities and nothing more verbose in the response tags.
          You are not verbose and your response doesn't include response out of context given to you. 
          If the question doesn't have a relevant answer in the givent content, you must answer "I don't have an answer" translated in {language} language.
          """
    elif task == 'summarization':
        sys_prompt = f"""\nGiven the content in {language} wrapped between <{language}> and </{language}> tags, summarize the content. 
          You respond in {language} language. You summarize to the point and not verbose.
          You are not verbose and your response doesn't include response out of context given to you. 
          If the content is not in {language}, you must answer "Please provide content in {language} language" translated in {language} language.
          """

    return sys_prompt

def trans_system_prompt(source, target, task):
    doer = 'transliterator' if task == 'transliterate' else 'translator'
    sys_prompt = f"Hello, You are an expert {doer}. For content provided in {source}, language perform {task} task to {target} language."
    sys_prompt = sys_prompt + f"""\n Given the content provided in {source} language wrapped between <{source}> and </{source}> tags, {task} the content to {target} language. 
    You are not verbose and your response doesn't include response out of context given to you. 
    If you don't know the {target} language or cannot perform {task}, you must answer "I don't have an answer" translated in {target} language.
    """
    return sys_prompt

def task_prompt_builder(content, language, task, tasktag):
    actual_question = [
                        {"role": "user", "content": f"""<{language}>{content}</{language}>
                        <{tasktag}>{task}</{tasktag}>"""}
                    ]
    return actual_question

def ask_question(llm, language, language_examples, content, question):
    actual_question = task_prompt_builder(content, language, question, 'question')
    message = language_examples + actual_question
    response_text = llm.invoke(
        system_prompt(language),
        message,
        max_tokens=1500,
    )
    if response_text:
        print(response_text[0].get('text'))

def summarize(llm, language, content):
    task = f"Summarize the content in {language}. Your response will be between the tags <summary> and </summary>."
    message = task_prompt_builder(content, language, task, 'summary') 
    response_text = llm.invoke(
        system_prompt(language, 'summarize'),
        message,
        max_tokens=1500,
    )
    if response_text:
        print(response_text[0].get('text'))
        
def transliterate(llm, source, target, content):
    task = f"Transliterate the content in {source} language to {target} language. Wrap the response between <{target}Transliteration> and </{target}Transliteration> tags."
    message = task_prompt_builder(content, source, task, 'transliterate') 
    response_text = llm.invoke(
        trans_system_prompt(source, target, 'transliterate'),
        message,
        max_tokens=1500,
    )
    if response_text:
        print(response_text[0].get('text'))
        
def translate(llm, source, target, content):
    task = f"Translate the content in {source} language to {target} language. Wrap the response between <{target}Translation> and </{target}Translation> tags."
    message = task_prompt_builder(content, source, task, 'translate') 
    response_text = llm.invoke(
        trans_system_prompt(source, target, 'translate'),
        message,
        max_tokens=1500,
    )
    if response_text:
        print(response_text[0].get('text'))

## Hindi Examples
### Question Answering

In [5]:
language = 'Hindi'
# few shot examples
hindi_examples = [
    {"role": "user", "content": f"""<{language}>लखनऊ के प्रसिद्ध चंद्रिका देवी मंदिर की महिमा और इतिहास बेहद अद्भुत है. मंदिर रामायण और महाभारत काल से जुड़ा हुआ है और यहाँ की वर्तमान शुरुआत 300 साल पुरानी है. देशभर से भक्त यहाँ अपनी मनोकामना लेकर आते हैं और देवी की शक्ति को नमन करते हैं. देखें वीडियो.</{language}>
    <question>Any location info present in the news?</question>"""},
    {"role": "assistant", "content": "<response>लखनऊ</response>"},
    {"role": "user", "content": f"""<question>What is famous in this news?</question>"""},
    {"role": "assistant", "content": "<response>चंद्रिका देवी मंदिर की महिमा और इतिहास</response>"},
]

In [6]:
content = """सारा अली खान स्टारर 'ऐ वतन मेरे वतन' अमेजन प्राइम पर स्ट्रीम हुई है. जो कि एक बायोपिक है. फिल्म की कहानी क्रांतिकारी उषा मेहता के जीवन पर आधारित है,  जिन्हें एक सच्ची गांधीवादी और कांग्रेस रेडियो की फाउंडर के रूप में भी जाना जाता है. बताते हैं कैसी है ये फिल्म और किरदार को जीवंत करने में सारा अली खान कितनी खरी उतर पाईं."""

In [7]:
question = "What is the content about?"
ask_question(llm, language, hindi_examples, content, question)

<response>यह सारा अली खान अभिनीत फिल्म 'ऐ वतन मेरे वतन' के बारे में है, जो क्रांतिकारी उषा मेहता की बायोपिक है।</response>


In [8]:
question = "Who is the person being discussed about?"
ask_question(llm,language, hindi_examples, content, question)

<response>उषा मेहता</response>


In [9]:
question = "Who is the actor being discussed about?"        
ask_question(llm,language, hindi_examples, content, question)

<response>सारा अली खान</response>


In [10]:
question = """
list all the named entities and their nature such as person, moviename, place, city, type etc in the format 
{'ent1': 'person', 'ent2': 'person', 'ent2': 'city','ent4': 'place', 'ent5':'celebrity'}
"""
ask_question(llm,language, hindi_examples, content, question)

<response>
{'सारा अली खान': 'person', 'उषा मेहता': 'person', 'ऐ वतन मेरे वतन': 'moviename', 'अमेजन प्राइम': 'place', 'कांग्रेस रेडियो': 'celebrity'}
</response>


In [11]:
question = "Which animal is mentioned?"        
ask_question(llm,language, hindi_examples, content, question)

<response>मैं नहीं देखता कि इस प्रदत्त सामग्री में किसी जानवर का उल्लेख है।</response>


### Summarization

In [12]:
content = """
इस बार राम नवमी न केवल अयोध्या के लिए बल्कि दुनिया भर के राम भक्तों के लिए बेहद खास है. 
सदियों बाद यह पहली राम नवमी जब भक्त अपने आराध्य का दर्शन भव्य राम मंदिर में दोपहर 12 बजे भगवान राम के बाल स्वरूप का सूर्य तिलक होगा. 
शुभ मुहूर्त में बालक राम का सूर्याभिषेक किया जाएगा. राम नवमी पर सूर्य तिलक के लिए सुबह 11 बजकर 05 मिनट से दोपहर 1 बजकर 35 मिनट तक शुभ मुहूर्त रहेगा.
"""
summarize(llm, language, content)

<summary>
यह राम नवमी बहुत खास है क्योंकि इस बार भक्त अपने आराध्य भगवान राम का दर्शन अयोध्या में बने नए भव्य राम मंदिर में कर सकेंगे। दोपहर 12 बजे बालक राम का सूर्याभिषेक किया जाएगा जो बहुत ही शुभ मुहूर्त है। यह पहली बार है जब सदियों बाद भक्तों को राम मंदिर में दर्शन करने का अवसर मिलेगा।
</summary>


### Translate

In [13]:
target_language = 'Marathi'
translate(llm, language, target_language, content)

<MarathiTranslation>
या वेळी रामनवमी फक्त अयोध्येसाठीच नव्हे तर जगभरातील रामभक्तांसाठी खूप विशेष आहे.
शेकडो वर्षांनंतर ही पहिली रामनवमी आहे जेव्हा भक्त त्यांच्या आराध्य देवतेचे दर्शन भव्य राममंदिरात दुपारी 12 वाजता भगवान रामाच्या बालस्वरूपाच्या सूर्यतिलकाचे करू शकतील.
शुभ मुहूर्तात बालक रामाचे सूर्याभिषेक केले जाईल. रामनवमीच्या दिवशी सूर्यतिलकासाठी सकाळी 11.05 ते दुपारी 1.35 या काळात शुभ मुहूर्त राहील.
</MarathiTranslation>


### Transliterate

Note: `anthropic.claude-3-sonnet-20240229-v1:0` is better at transliteration.

In [14]:
target_language = 'English'
transliterate(llm, language, target_language, content)

<EnglishTransliteration>
Is baar Raam Navmi na kevala Ayodhya ke lie balki duniya bhar ke Raam bhakton ke lie behad khaas hai.
Sadiyon baad yeh pehli Raam Navmi jab bhakt apne aaradhya ka darshan bhavya Raam Mandir mein dopahar 12 baje Bhagavaan Raam ke baal swaroop ka soory tilak hoga.
Shubh muhoort mein baalak Raam ka sooryaabhishek kiya jaayega. Raam Navmi par soory tilak ke lie subah 11 bajekar 05 minute se dopahar 1 bajekar 35 minute tak shubh muhoort rahega.
</EnglishTransliteration>


## Telugu Examples
### Question Answering

In [15]:
language = 'Telugu'

In [16]:
content = """
ఎన్‌పీసీఐకు సంబంధించిన తాజా ప్రకటనలో క్రెడిట్ కార్డు ద్వారా ఈఎంఐ సదుపాయం, క్రెడిట్ ఖాతా బిల్లు చెల్లింపు, వాయిదా చెల్లింపు ఎంపికలు, పరిమితి నిర్వహణ కార్యాచరణల వంటి అనేక కీలక సదుపాయాలను అందిస్తున్నాయి. 
మే 31, 2024లోపు ఈ ఫీచర్లను అమలు చేయాలని బ్యాంకులు, కార్డు జారీచేసే వారితో సహా జారీ చేసే సంస్థలు ఆదేశించింది.
జూన్ 30 2024 లోగా అన్ని వివరాలు ఆధార్ కార్డ్ లొ పొందు పరచాలి.
"""
telugu_examples = []

In [17]:
question = "What is the content about?"
ask_question(llm,language, telugu_examples, content, question)

<response>ఈ విషయం క్రెడిట్ కార్డ్‌లకు సంబంధించిన కొత్త సదుపాయాలు, ఫీచర్లపై మాట్లాడుతుంది మరియు ఈ కొత్త కార్యాచరణలను అమలు చేయడానికి గడువు వరకు పేర్కొంది</response>


In [18]:
question = "Who is the person being discussed about?"
ask_question(llm,language, telugu_examples, content, question)

<response>సార్వజనిక బ్యాంకులు, కార్డు జారీచేసే వారు</response>


In [19]:
question = "Who is the actor being discussed about?"        
ask_question(llm,language, telugu_examples, content, question)

<response>నేను ఈ సమాచారంలో నటుడి గురించి ఏమీ చూడలేదు</response>


In [20]:
question = "Is there any benefit described in news?"        
ask_question(llm,language, telugu_examples, content, question)

<response>అవును, ప్రకటనలో క్రెడిట్ కార్డు ద్వారా ఈఎంఐ సదుపాయం, క్రెడిట్ ఖాతా బిల్లు చెల్లింపు, వాయిదా చెల్లింపు ఎంపికలు, పరిమితి నిర్వహణ కార్యాచరణల వంటి అనేక కీలక సదుపాయాలను అందిస్తున్నట్లు తెలుపబడింది</response>


In [21]:
question = "Any dates or deadlines? respond with {'DD-MM-YYYY': [action1, action2, ..], 'DD-MM-YYY': [action1]}"        
ask_question(llm,language, telugu_examples, content, question)

<response>
{'31-05-2024': ['క్రెడిట్ కార్డు ద్వారా ఈఎంఐ సదుపాయం, క్రెడిట్ ఖాతా బిల్లు చెల్లింపు, వాయిదా చెల్లింపు ఎంపికలు, పరిమితి నిర్వహణ కార్యాచరణలు అమలు చేయాలి'],
 '30-06-2024': ['అన్ని వివరాలు ఆధార్ కార్డ్ లొ పొందు పరచాలి']}
</response>


### Summarization

In [22]:
summarize(llm, language, content)

<summary>
ఎన్పీసీఐ క్రెడిట్ కార్డుల కోసం క్రెడిట్ ఖాతా చెల్లింపు, వాయిదా చెల్లింపు, ఈఎంఐ సదుపాయం, పరిమితి నిర్వహణ వంటి కీలక సదుపాయాలను పరిచయం చేసింది. బ్యాంకులు, కార్డు జారీ చేసే సంస్థలు మే 31, 2024 నాటికి ఈ ఫీచర్లను అమలు చేయాలని, జూన్ 30, 2024 నాటికి ఆధార్ కార్డులో అన్ని వివరాలను పొందుపరచాలని ఆదేశించింది.
</summary>


### Translate

In [23]:
content = "కృష్ణుడు తన స్నేహితుల కోసం పూలవృక్షాలను నాటాడు."
target_language = 'English'
translate(llm, language, target_language, content)

<EnglishTranslation>Krishna planted flower trees for his friends.</EnglishTranslation>


### Transliteration

Transliteration is the process of representing or intending to represent a word, phrase, or text in a different script or writing system

In [24]:
target_language = 'English'
transliterate(llm, language, target_language, content)

<EnglishTransliteration>Krishna nATADu tana snehitula kosam pUlavrkshAlanu nATADu.</EnglishTransliteration>
